In [1]:
''' This cell contains all constants thay may differ on our machines '''

ELASTIC_HOST = 'localhost'
ELASTIC_PORT = 9200
COLLECTION_DIRECTORY = "../byweb" # directory with .out files to process
COLLECTION_DIRECTORY_MYSTEM = "../byweb_stem" # directory with .out files after mystem processing

QUERIES_FILE = "../web2008_adhoc.xml"
RELEVANCE_FILE = "../relevant_table_2009.xml"

In [2]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time

In [3]:
es = Elasticsearch([{'host': ELASTIC_HOST, 'port': ELASTIC_PORT, 'timeout': 360, 'maxsize': 25}])

In [4]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'title': {
                'type' : 'text'
            }
        }
    }
}

In [7]:
def recreate_index(index_name):
    try:
        es.indices.delete(index=index_name)
    except:
        pass
    es.indices.create(index=index_name, body=settings)

In [8]:
recreate_index('hw2index')
# recreate_index('hw2index_stem')

In [9]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [10]:
class Document:
    def __init__(self, doc_url, doc_id, sz_bytes, sz_words):
        self.url = doc_url       # document url
        self.id = doc_id         # unique document id (str)
        self.sz_bytes = sz_bytes # document size in bytes before deleting html markup
        self.sz_words = sz_words # number of words in document before deleting html markup
        self.words = []          # list of words in document after deleting html markup
        self.links = []          # lisk of links in document

In [11]:
import os
from tqdm import tqdm
from tqdm import tqdm_notebook
import pickle

class BaseDocumentProcessor:
    def process(self, document):
        pass
    def result(self):
        pass

def process_file(d, f, processor, pbar):
    print("processing", os.path.join(d, f))
    with open(os.path.join(d, f), "rb") as fin, open(os.path.join(d, f.replace(".out", ".title")), "rb") as tfin:
        dct = pickle.load(tfin)
        while True:
            pbar.update(1)
            try:
                document = pickle.load(fin)
            except:
                break
            processor.process(document, dct[document.id])

def process_collection(directory, processor):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):
            process_file(directory, file, processor, pbar)

In [ ]:
document_urls = {}

class GetDocUrls(BaseDocumentProcessor):
    def __init__(self):
        """ do all initialization here """
    
    def process(self, document):
        """ document: Document (see first cell)
            process each document here """
        document_url = str(document.url)[2:-1].split("?")[0].split("#")[0]
        if document_url[-1] == '/':
            document_url = document_url[:-1]
        document_urls[document_url] = True
        
    def result(self):
        pass
        
processor = GetDocUrls()        
process_collection(COLLECTION_DIRECTORY, processor)
processor.result()

In [ ]:
from tqdm import tqdm_notebook
import networkx as nx
import operator
pagerank_dict = {}

class GraphBuider(BaseDocumentProcessor):
    def __init__(self):
        """ do all initialization here """
        self.edge_list = []
    
    def process(self, document):
        """ document: Document (see first cell)
            process each document here """
        links = document.links
        document_url = str(document.url)[2:-1].split("?")[0].split("#")[0]
        if document_url[-1] == '/':
            document_url = document_url[:-1]
        links_absolute_no_params = []
        for link in links:
            if not link:
                continue
            link_absolute_path = link    
            if not (link_absolute_path.startswith("http://") or link_absolute_path.startswith("https://")):
                if link_absolute_path[0] == '.':
                    link_absolute_path = link_absolute_path[2:]
                link_absolute_path = document_url.rsplit("/", 1)[0] + "/" + link_absolute_path
            link_absolute_no_params = link_absolute_path.split("?")[0].split("#")[0]
            if link_absolute_no_params[-1] == '/':
                    link_absolute_no_params = link_absolute_no_params[:-1]
            if not (link_absolute_no_params in links_absolute_no_params):
                links_absolute_no_params.append(link_absolute_no_params)
        for link in links_absolute_no_params:
            if link in document_urls:
                self.edge_list.append((document_url, link))
        
    def result(self):
        graph = nx.DiGraph(self.edge_list)
        return nx.pagerank(graph)
        
processor = GraphBuider()        
process_collection(COLLECTION_DIRECTORY, processor)
pagerank_dict = processor.result()

In [15]:
class IndexDocs(BaseDocumentProcessor):
    def __init__(self, index_name):
        """ do all initialization here """
        self.actions = []
        self.index_name = index_name        
    
    def process(self, document, title):
        """ document: Document (see first cell)
            process each document here """
        document_url = str(document.url)[2:-1].split("?")[0].split("#")[0]
        if document_url[-1] == '/':
            document_url = document_url[:-1]
        pagerank = 0     
#         if document_url in pagerank_dict.keys():
#             pagerank = max(0.00005, pagerank_dict[document_url])
        self.actions.append(create_es_action(self.index_name, document.id, json.dumps({'content' : document.words, 'title' : title})))
        
    def result(self):
        return self.actions
        


In [13]:
def es_actions_generator(index_name, collection_directory):
    processor = IndexDocs(index_name)
    start = time.time()
    process_collection(collection_directory, processor)
    end = time.time()
    print("preprocess time = ", end - start)
    return processor.result()

In [16]:
import time
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator('hw2index', COLLECTION_DIRECTORY), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))



  0%|          | 377/200000 [00:00<00:53, 3746.09it/s]


processing ../byweb/byweb.0.out


 10%|█         | 20764/200000 [00:04<00:42, 4256.03it/s]

processing ../byweb/byweb.9.out



 20%|██        | 40704/200000 [00:08<00:35, 4520.28it/s]

...

processing ../byweb/byweb.4.out



100%|█████████▉| 199771/200000 [00:41<00:00, 4605.01it/s]

preprocess time =  41.25140118598938



200010it [00:53, 4605.01it/s]                            


Time=178.12326383590698


In [ ]:
import time
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator('hw2index_stem', COLLECTION_DIRECTORY_MYSTEM), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))

In [ ]:
import requests
param = (('v', ''),) # '-v' is for --verbose

# call the class's method to get an HTTP response model
resp = requests.get(f'http://{ELASTIC_HOST}:{ELASTIC_PORT}/_cat/indices', params=param)
resp.text

In [17]:
from bs4 import BeautifulSoup

def read_queries():
    queries = {}
    with open(QUERIES_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            queries[task["id"]] = task.text
    return queries

def read_relevance():
    relevance = {}
    with open(RELEVANCE_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            rel = [doc["id"] for doc in task.find_all("document") if doc["relevance"] == "vital"]
            if rel:
                relevance[task["id"]] = rel
    return relevance

In [18]:
relevance = read_relevance()
queries = read_queries()

In [19]:
def build_query(query):
    return {
            'query': {
                'bool': {
                    'should': [
                        {
                            'match': {
                                'content': {
                                    'query': query,
                                    'boost': 1
                                }
                            }
                        },
                        {
                            'match': {
                                'title': {
                                    'query': query,
                                    'operator': 'OR',
                                    'boost': 0.3
                                }
                            }
                        },
                    ]
                }
            }
        }

def run_search(query, size, index_name):
    search_result = es.search(index=index_name, body=build_query(query), size=size)['hits']
    return [hit['_id'] for hit in search_result['hits']]

def count_metrics(index_name, queries):
    p = 0
    r = 0
    rp = 0
    ma_p = 0
    n = 0
    f1 = {}
    for qid, query in queries.items():
        if qid not in relevance.keys():
            continue
        n += 1
        rel = relevance[qid]
        relevant = len(rel)
        hits = run_search(query, max(20, relevant), index_name)
        hits_r = hits[:relevant]
        hits_20 = hits[:20]
        true_positive_20 = sum(1 for did in hits_20 if did in rel)
        true_positive_r = sum(1 for did in hits_r if did in rel)
        pp = true_positive_20 / 20
        rr = true_positive_20 / relevant
        p += pp
        r += rr
        rp += true_positive_r / relevant
        ma_p += sum(sum(1 for did in hits[:k] if did in rel) / k for k in range(1, 21)) / 20
        f1[qid] = 0 if pp * rr == 0 else pp * rr / (pp + rr)
    print("p@20:", p / n)
    print("r@20:", r / n)
    print("r-precision:", rp / n)
    print("map@20:", ma_p / n)
    return f1

In [20]:
f1 = count_metrics('hw2index', queries)
# f1_stem = count_metrics('hw2index_stem', stem_queries(queries))

# diff = sorted([(queries[qid], f1_stem[qid], f1[qid], f1_stem[qid] - f1[qid]) for qid in f1.keys()], key=lambda tup: -tup[-1])
# print(*diff[:5], sep="\n")

p@20: 0.351717171717172
r@20: 0.22831787409824356
r-precision: 0.28625719694853574
map@20: 0.4146079284488261


In [ ]:
class MystemProcessor(BaseDocumentProcessor):
    def __init__(self, fout):
        self.m = Mystem(grammar_info=False, disambiguation=False)
        reg = re.compile('[a-zа-яё0-9\-]')
        self.filterFunc = lambda w : reg.match(w)
        self.fout = fout

    def process(self, document):
        text = ' '.join(document.words).lower()
        lemmas = self.m.lemmatize(text)
        words = list(filter(self.filterFunc, lemmas))
        doc = Document(document.url, document.id, document.sz_bytes, document.sz_words)
        doc.words = words
        doc.links = document.links
        pickle.dump(doc, self.fout)

         
    def result(self):
        pass

def process_collection_with_mystem(directory, outdirectory):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):        
            with open(os.path.join(outdirectory, file), "wb") as fout:
                processor = MystemProcessor(fout)
                process_file(directory, file, processor, pbar)

In [ ]:
process_collection_with_mystem(COLLECTION_DIRECTORY, COLLECTION_DIRECTORY_MYSTEM)

In [ ]:
def stem_queries(queries):
    m = Mystem(grammar_info=False, disambiguation=False)
    reg = re.compile('[a-zа-яё0-9\-]')
    filterFunc = lambda w : reg.match(w)
    result = {}
    for (qid, text) in queries.items():
        result[qid] = ' '.join(filter(filterFunc, m.lemmatize(text)))
    return result

In [ ]:
queries = stem_queries(queries)
print(queries)